This notebook will work with a MATLAB kernel installed: 
https://github.com/Calysto/matlab_kernel
(it's kind of janky in that the kernel dies often, tips for a better one?)

#### Step 1 - rearrange the 9 task Features in the position they were presented on screen

Replace the relevant variables, Stimuli and Row_Order, with dummy ones (too lazy to load the full datafile..)

In [ ]:
Stimuli_Trial(:,:,1) = [2 1 3];
Stimuli_Trial(:,:,2) = [1 2 3];
Stimuli_Trial(:,:,3) = [3 1 2];

Stimuli_Trial

Row_Order_Trial = [2 1 3];

Assuming you have the `eyelinkPreproc.m` file (let me know if not) the relevant lines are: 

334: do `squeeze` on the Stimuli variable on that trial, which will remove the trial dimension and return a 3x3 array of Dimensions x Features. The way to be sure you got that back is to note that within each Dimension you have one of each Feature. In this variable, the first Fimension (F) is always on row 1, the second (H) is always on row 2 and the third (T) is always on row 3. 

In [2]:
S = squeeze(Stimuli_Trial)';
S


S =

     2     1     3
     1     2     3
     3     1     2



335 & 337: map to 1-9 Feature notation. 1-3 are Faces, 4-6 are Houses, 7-9 are Tools. 

In [ ]:
hlp = [zeros(1,3); 3*ones(1,3); 6*ones(1,3)];

F = S+hlp; 
F

Error using eval
Undefined f

339-341: use Row_Order to remap Dimensions to their correct position. This step will give you back what the subject actually saw on the screen. 

In [ ]:
Features = NaN(3,3);
for i = 1:3           
    Features(i,:) = (F(find(Row_Order_Trial == i),:));
end

Features

This says, House1 was on the top left, House2 was in the top center, House3 was in the top right, Face2 was in the center left, and so on. I've found it easier to think about the 1-9 coding, and just remember that 1-3 is Dimension 1, 4-6 is Dimension 2, 7-9 is Dimension 3.

#### Step 2 - loop through events in each trial and use AOIs to assign each eyetracking event to a task feature

This happens in a separate function, `event2stims`, which starts on line 624. There are two AOI styles, row-wise AOI (3 rectangular rows, one per Dimension) and feature-wise AOI (9 rectangular squares, one per Feature). Take a look at lines 681-724 to see how the mapping happens for the feature-wise AOI (don't judge the MATLAB code :). Here, for each event that fell inside the AOIs, we're looping through 9 AOIs and asking whether the location of the eye position during that event fell in that AOI. If so, we use the Features matrix computed above to assign the correct Feature and Dimension to that event. 